In [12]:
"""
    Who: Ericson Mattoso
    When: 04/nov/2021
    What: This script is responsable to take each house zipcode and find the coordinates for it.
    Why: Coordinates are necessary to print the addesses on our map
    Where: Zipcode comes from Pararius.nl and Coordinates comes from postcodebijadres.nl
    How: scrapping data through BeautifulSoup
"""
# Libs
from bs4 import BeautifulSoup
import glob
import os
import pandas as pd
import requests

# list all files
list_of_files = glob.glob("../data/temp/df_pararius_*.csv")
# get the newest
latest_file = max(list_of_files, key=os.path.getctime)
# read the newest file
df_pararius = pd.read_csv(latest_file, index_col=[0])
# get all zipcodes in our db
df_zipcode = pd.read_csv('../data/raw/zipcode.csv', index_col=[0])
# what are the zipcodes we have scrapped?
temp1 = df_pararius['postcode'].unique()
# what are those we already have the coordinates?
temp2 = df_zipcode['postcode'].unique()
# what is new?
cep_zip = list(set(temp1) - set(temp2))
# how many new zipcodes without coordidates we have
print(len(cep_zip))
cep_zip = ['2274PC']

def postnl_scrape(cep):
    """
        This function shall request coordinates from postcodebijadres
    """
    # creates link
    url = 'https://postcodebijadres.nl/' + cep
    # create header to simulate human behaviour on scrape
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"}
    # scrape
    soup = BeautifulSoup(requests.get(
        url, headers=headers).content, "html.parser")
    # html as string
    return soup


# empty variables
zipcode = pd.DataFrame({})
description_1 = {}

for cep in cep_zip:
    soup = postnl_scrape(cep)
    try:
        split0 = soup.find_all('table', {"class": "table table-bordered mt-3"})[0]
        split1 = soup.find_all('table', {"class": "table table-bordered mt-3"})[1]
        lat = split1.find_all('td')[1].text
        lon = split1.find_all('td')[3].text
        description_1.update({'latitude': lat,'longitude': lon})
        for num in range(9):
            split0_key = split0.find_all('th')[num].text
            split0_val = split0.find_all('td')[num].text
            description_1.update({split0_key: split0_val})
            print(description_1)
        zipcode_ = pd.DataFrame(description_1, index=[0])
        zipcode = pd.concat([zipcode, zipcode_], 0)

        #zipcode.to_csv(f'data/temp/cep/zipcode_{cep}.csv')
        print(cep)
    except:

        print('#####################################', cep)
        pass


15
{'latitude': '52.076370204397', 'longitude': '4.3430583854467', 'Postcode': '2274 PC'}
{'latitude': '52.076370204397', 'longitude': '4.3430583854467', 'Postcode': '2274 PC', 'Straat': '\nvan Naeltwijckstraat 48 - 84 \n\n'}
{'latitude': '52.076370204397', 'longitude': '4.3430583854467', 'Postcode': '2274 PC', 'Straat': '\nvan Naeltwijckstraat 48 - 84 \n\n', 'Netnummer': '070'}
{'latitude': '52.076370204397', 'longitude': '4.3430583854467', 'Postcode': '2274 PC', 'Straat': '\nvan Naeltwijckstraat 48 - 84 \n\n', 'Netnummer': '070', 'Postcodenummer': '2274'}
{'latitude': '52.076370204397', 'longitude': '4.3430583854467', 'Postcode': '2274 PC', 'Straat': '\nvan Naeltwijckstraat 48 - 84 \n\n', 'Netnummer': '070', 'Postcodenummer': '2274', 'Buurt': 'Voorburg Noord'}
{'latitude': '52.076370204397', 'longitude': '4.3430583854467', 'Postcode': '2274 PC', 'Straat': '\nvan Naeltwijckstraat 48 - 84 \n\n', 'Netnummer': '070', 'Postcodenummer': '2274', 'Buurt': 'Voorburg Noord', 'Wijk': 'Wijk 11 V

<ipython-input-12-038e3a8b3ed2>:68: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode = pd.concat([zipcode, zipcode_], 0)
